In [1]:
import glob
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
path = '../Downloads/CSVs/' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    print(file_)
    try:
        df = pd.read_csv(file_,index_col=None, header=0)
    except:
        df = pd.read_csv(file_,index_col=None, header=0, encoding = 'latin1')
    list_.append(df)
frame = pd.concat(list_)

../Downloads/CSVs/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv


/Users/ozalmanson/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


../Downloads/CSVs/Monday-WorkingHours.pcap_ISCX.csv


/Users/ozalmanson/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


../Downloads/CSVs/Friday-WorkingHours-Morning.pcap_ISCX.csv
../Downloads/CSVs/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
../Downloads/CSVs/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv


/Users/ozalmanson/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20,21,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


../Downloads/CSVs/Tuesday-WorkingHours.pcap_ISCX.csv
../Downloads/CSVs/Wednesday-workingHours.pcap_ISCX.csv
../Downloads/CSVs/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv


/Users/ozalmanson/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,3,6,20,21,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
import numpy as np

In [5]:
frame = frame.dropna(how = 'all')

In [6]:
frame[' Flow Packets/s'] = frame[' Flow Packets/s'].astype('float64')
frame['Flow Bytes/s'] = frame['Flow Bytes/s'].astype('float64')
frame = frame.drop(' Timestamp', axis = 1)
frame = frame.replace([np.inf, -np.inf], np.nan)

In [7]:
#frame = frame.merge(dest_ip_num, left_index = True, right_index = True)
#frame = frame.merge(src_ip_num, left_index = True, right_index = True)

In [8]:
non_trainable = ['Flow ID', ' Source IP', ' Destination IP', ' Timestamp', 'External IP', ' Label', 'dest_a','dest_b','dest_c','dest_d','src_a','src_b', 'src_c','src_d']

In [9]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

/Users/ozalmanson/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Multi Class LSTM

##### creating the multi class labels as one-hot encoding

In [10]:
label_mapping = {val: i for i, val in enumerate(list(frame[' Label'].drop_duplicates()))}
y_all = frame[' Label'].replace(label_mapping)
y_all = y_all.as_matrix()

In [11]:
y_all = to_categorical(y_all)

In [12]:
frame_mat = frame[[col for col in frame.columns if col != ' Label' and col not in non_trainable]].as_matrix()
frame_tsteps = frame_mat.reshape((frame_mat.shape[0], 1, frame_mat.shape[1]))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(frame_tsteps, y_all, test_size=0.2, shuffle = False, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
scaler = StandardScaler()

In [16]:
scaled_train_X = scaler.fit_transform(np.nan_to_num(X_train.reshape((X_train.shape[0], X_train.shape[2]))))

In [17]:
scaled_test_X = scaler.transform(np.nan_to_num(X_test.reshape((X_test.shape[0], X_test.shape[2]))))

In [18]:
y_test.shape

(566149, 15)

In [19]:
scaled_train_X = scaled_train_X.reshape((scaled_train_X.shape[0], 1, scaled_train_X.shape[1]))
scaled_test_X = scaled_test_X.reshape((scaled_test_X.shape[0], 1, scaled_test_X.shape[1]))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from matplotlib import pyplot
# design network
model = Sequential()
model.add(LSTM(64, input_shape=(scaled_train_X.shape[1], scaled_train_X.shape[2])))
model.add(Dense(32, activation = 'relu'))
#model.add(Dense(32, activation = 'relu'))
model.add(Dense(y_train.shape[1],  activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())
# fit network
history = model.fit(scaled_train_X, y_train, epochs=10, batch_size=72, validation_data=(scaled_test_X, y_test), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [86]:
pred = model.predict(scaled_test_X)

In [87]:
from sklearn.metrics import classification_report

In [99]:
print(classification_report(np.argmax(y_test, axis = 1), np.argmax(pred, axis =1)))

             precision    recall  f1-score   support

          0       0.94      1.00      0.97    530659
          9       0.00      0.00      0.00     23006
         10       0.00      0.00      0.00     10293
         11       0.00      0.00      0.00        11
         12       0.00      0.00      0.00      1507
         13       0.00      0.00      0.00       652
         14       0.00      0.00      0.00        21

avg / total       0.88      0.94      0.91    566149



/Users/ozalmanson/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [100]:
from sklearn.metrics import confusion_matrix

In [102]:
print(confusion_matrix(np.argmax(y_test, axis = 1), np.argmax(pred, axis =1)))

[[530659      0      0      0      0      0      0]
 [ 23006      0      0      0      0      0      0]
 [ 10293      0      0      0      0      0      0]
 [    11      0      0      0      0      0      0]
 [  1507      0      0      0      0      0      0]
 [   652      0      0      0      0      0      0]
 [    21      0      0      0      0      0      0]]


## Binary Timesteps LSTM

##### Creating binary label

In [13]:
y_binary = frame[' Label'].apply(lambda val: 1 if val != 'BENIGN' else 0).as_matrix()

##### creating timesteps dataset

In [14]:
seq_length = 30
dataX = []
dataY = []

In [15]:
dataX = [frame_mat[i:i + seq_length] for i in range(len(frame_mat) - seq_length)]

In [16]:
dataY = [y_binary[i + seq_length - 1] for i in range(len(frame_mat) - seq_length)]

In [17]:
data_X_tsteps = np.array(dataX)

In [18]:
data_y_tsteps = np.array(dataY)

##### dataset shape is (Samples, Timesteps, Features)

In [19]:
data_X_tsteps.shape

(2830713, 30, 80)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data_X_tsteps, data_y_tsteps, test_size=0.2, shuffle = False, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from matplotlib import pyplot
# design network
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(32, activation = 'relu'))
#model.add(Dense(32, activation = 'relu'))
model.add(Dense(1,  activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')
print(model.summary())
# fit network
history = model.fit(X_train, y_train, epochs=5, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                37120     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 39,233
Trainable params: 39,233
Non-trainable params: 0
_________________________________________________________________
None
Train on 2264570 samples, validate on 566143 samples
Epoch 1/5


In [133]:
print(classification_report(y_test_binary, list(map(lambda val: 1 if val > 0.5 else 0,pred))))

             precision    recall  f1-score   support

          0       0.98      0.96      0.97    530659
          1       0.57      0.73      0.64     35490

avg / total       0.96      0.95      0.95    566149



In [158]:
print(confusion_matrix(y_test_binary, list(map(lambda val: 1 if val > 0.5 else 0,pred))))

[[511462  19197]
 [  9592  25898]]
